# Mini Project: Monte Carlo Simulation for Financial Risk Assessment

## Objective

This mini-project will guide students through the implementation of a Monte Carlo simulation to estimate financial risk using advanced random number generation techniques. By completing this project, students will apply pseudo-random and quasi-random numbers, variance reduction methods, and numerical approximations to assess the Value at Risk (VaR) of a portfolio.

## Problem Statement

You are a quantitative analyst working for a financial institution. Your task is to estimate the Value at Risk (VaR) of a stock portfolio using Monte Carlo simulations. The risk estimation should incorporate techniques such as:

- Pseudo-random and quasi-random number generation
- Antithetic variance reduction
- Taylor series approximations
- Convergence analysis

## Steps to Complete the Project



### Step 1: Data Initialization

1. Simulate the daily return of a stock using a normal distribution with a given mean and standard deviation.
2. Implement a pseudo-random number generator (PRNG) to generate these stock returns.
3. Compare the output with a quasi-random number generator (QRNG) using a Sobol sequence.


The formula for daily returns is:
 $$r_t = \mu + \sigma \cdot Z$$
Where:
* $r_t =$ daily return at time $t$
* $\mu =$ mean return
* $\sigma =$ standard deviation of returns
* $Z =$ random variable form a standard normal distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc, norm

mu = 0.001  # Mean daily return 
sigma = 0.02  # Standard deviation of daily returns 
n_samples = 1000  # Number of samples to generate

# Daily returns - PRNG 
prng_returns = np.random.normal(mu, sigma, n_samples)

# Daily returns - QRNG 
sobol_engine = qmc.Sobol(d=1, scramble=True)  # d=1 for 1-dimensional sequence
sobol_samples = sobol_engine.random(n_samples)
# Transform Sobol samples to normal distribution using inverse CDF
qrng_returns = mu + sigma * norm.ppf(sobol_samples).flatten()

# Compare 
plt.figure(figsize=(12, 6))

# Plot PRNG returns
plt.subplot(1, 2, 1)
plt.hist(prng_returns, bins=50, density=True, alpha=0.6, color='blue', label='PRNG Returns')
plt.title('PRNG Returns')
plt.xlabel('Daily Return')
plt.ylabel('Density')
plt.legend()

# Plot QRNG returns
plt.subplot(1, 2, 2)
plt.hist(qrng_returns, bins=50, density=True, alpha=0.6, color='green', label='QRNG Returns')
plt.title('QRNG Returns (Sobol Sequence)')
plt.xlabel('Daily Return')
plt.ylabel('Density')
plt.legend()

plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'numpy'

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(prng_returns, bins=50, density=True, histtype='step', color='blue', label='PRNG Returns')
plt.hist(qrng_returns, bins=50, density=True, histtype='step', color='green', label='QRNG Returns')
plt.title('Comparison of PRNG and QRNG Returns (Border Lines)')
plt.xlabel('Daily Return')
plt.ylabel('Density')
plt.legend()
plt.show()


### Step 2: Monte Carlo Simulation for Stock Price Evolution

1. Simulate stock price paths using the Geometric Brownian Motion (GBM) model:

   
   $$S_t = S_0 \exp\left( (\mu - \frac{\sigma^2}{2}) t + \sigma W_t \right)$$
   

   where $W_t$ is a Wiener process.

2. Generate multiple stock price paths using both PRNG and QRNG methods.
3. Visualize the simulated stock price paths.




In [ ]:
S0 = 100  # Stock price
T = 1  # 1 year
n_steps = 252  # trading days in a year
n_paths = 10  # Paths 

t = np.linspace(0, T, n_steps)

# Simulate stock price - GBM
def simulate_gbm(S0, mu, sigma, t, n_paths, use_qrng=False):
    n_steps = len(t)
    dt = t[1] - t[0]  # Time step size
    paths = np.zeros((n_steps, n_paths))
    paths[0, :] = S0 

    if use_qrng:
        # Sobol for QRNG
        sobol_engine = qmc.Sobol(d=n_steps-1, scramble=True)
        sobol_samples = sobol_engine.random(n_paths)
        Z = norm.ppf(sobol_samples).T  # Transform to standard normal and transpose to match dimensions
    else:
        # PRNG
        Z = np.random.normal(0, 1, (n_steps-1, n_paths))

    for i in range(1, n_steps):
        drift = (mu - 0.5 * sigma**2) * dt
        diffusion = sigma * np.sqrt(dt) * Z[i-1, :]
        paths[i, :] = paths[i-1, :] * np.exp(drift + diffusion)

    return paths

# PRNG
prng_paths = simulate_gbm(S0, mu, sigma, t, n_paths, use_qrng=False)

# QRNG
qrng_paths = simulate_gbm(S0, mu, sigma, t, n_paths, use_qrng=True)

plt.figure(figsize=(12, 6))

# PRNG 
plt.subplot(1, 2, 1)
for i in range(n_paths):
    plt.plot(t, prng_paths[:, i], lw=1)
plt.title('Stock Price Paths (PRNG)')
plt.xlabel('Time (Years)')
plt.ylabel('Stock Price')
plt.grid(True)

# Plot QRNG paths
plt.subplot(1, 2, 2)
for i in range(n_paths):
    plt.plot(t, qrng_paths[:, i], lw=1)
plt.title('Stock Price Paths (QRNG - Sobol)')
plt.xlabel('Time (Years)')
plt.ylabel('Stock Price')
plt.grid(True)

plt.tight_layout()
plt.show()

### Step 3: Estimating Value at Risk (VaR)

1. Compute portfolio returns from the simulated stock prices.
2. Estimate the 95% and 99% VaR using both historical simulation and Monte Carlo methods.
3. Implement antithetic variance reduction to improve estimation accuracy.
4. Compare VaR estimates using pseudo-random and quasi-random methods.



### Step 4: Taylor Series Approximation for Risk Adjustment

1. Use a second-order Taylor series expansion to approximate the portfolio's risk exposure.
2. Analyze the effect of approximation on risk estimation.
3. Compare results with the direct computation approach.



### Step 5: Convergence Analysis and Final Report

1. Study the convergence behavior of Monte Carlo estimates by increasing the number of simulations.
2. Plot error vs. number of simulations to assess efficiency.
3. Write a summary report discussing findings, methodology, and computational performance.



## Rubric (Total: 100 Points)

| Criteria | Excellent (20) | Good (15) | Fair (10) | Poor (5) |
|----------|--------------|----------|---------|-------|
| **Step 1: Data Initialization** | PRNG and QRNG correctly implemented and analyzed | Minor issues in implementation | Incomplete or incorrect implementation | Little to no effort |
| **Step 2: Monte Carlo Stock Simulation** | GBM correctly simulated and visualized | Small inaccuracies | Major issues in simulation | No proper implementation |
| **Step 3: VaR Estimation** | VaR correctly computed with all methods | Minor computational errors | Missing one key method | Poor implementation |
| **Step 4: Taylor Series Approximation** | Taylor expansion correctly used and compared | Some approximation issues | Incorrect implementation | Not attempted |
| **Step 5: Convergence Analysis & Report** | Convergence studied with clear insights | Minor inconsistencies in analysis | Weak discussion | No analysis provided |

## Submission Guidelines

- Submit a Jupyter Notebook (`.ipynb`) with well-commented code and visualizations.
- Include a short report summarizing findings and observations.
- Ensure reproducibility by setting random seeds where applicable.

## Bonus Challenge

- Implement a risk-adjusted portfolio optimization strategy using Monte Carlo results.
- Compare performance with standard VaR estimation.

This project provides a structured, hands-on experience in financial risk modeling using Monte Carlo simulations. It integrates multiple numerical techniques, ensuring practical application and computational efficiency.